In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [ ]:
from matplotlib.pyplot import rcParams

rcParams['figure.figsize'] = 15, 10
rcParams["font.weight"] = "bold"
rcParams["axes.labelweight"] = "bold"
rcParams["font.size"] = 12

In [ ]:
df_placas = pd.read_csv('placas.csv')

df_placas

In [ ]:
df_placas.isna().sum() # Verificando los valores nulos

In [ ]:
df_placas.info() # Verificando los tipos de datos

In [ ]:
df_placas["MODULE_TEMPERATURE"].isna()

In [ ]:
df_placas["MODULE_TEMPERATURE"].isna().sum()

In [ ]:
df_placas["MODULE_TEMPERATURE"][df_placas["MODULE_TEMPERATURE"].isna()]

In [ ]:
df_placas[20:330][["MODULE_TEMPERATURE", "AMBIENT_TEMPERATURE", "IRRADIATION"]]

In [ ]:
df_placas.drop(df_placas.dropna().index)

In [ ]:
# No hay nada que limpiar

df_placas.shape[0] - 0

In [ ]:
df_solar_filt = df_placas.dropna().reset_index(drop=True)
df_solar_filt

### Análisis exploratorio, tratamiento y limpieza de datos


In [ ]:
df_solar_filt.describe()

In [ ]:
# consultar el tipo de datos
df_solar_filt.dtypes

In [ ]:
df_solar_filt.dtypes[df_solar_filt.dtypes == "object"]

In [ ]:

df_solar_filt['DATE_TIME'].value_counts()

In [ ]:
# columnas a eliminar por ser informativas
columnas_drop = ["DATE_TIME", "SOURCE_KEY"]

In [ ]:
df_solar_filt2 = df_solar_filt.drop(columns = columnas_drop)
print("Tamaño del tablón filtrado: ", df_solar_filt.shape)
print("Tamaño del tablón nuevo: ", df_solar_filt2.shape)

In [ ]:
df_solar_filt2.dtypes.value_counts()

### Reducción de variables
Importancia de variables

In [ ]:
# generar la lista de features y la variable target
target = 'MODULE_TEMPERATURE'
features = [x for x in df_solar_filt2.columns if x != target]

print(target)
print(features)

In [ ]:
df_solar_filt2[target]

In [ ]:
# Importar el algoritmo de árboles de decisión
from sklearn.tree import DecisionTreeRegressor

# Asignar el algortimo e indicar la profundidad máxima del árbol (con un número rotandamente grande para sobreajustar)
arbol_importancia = DecisionTreeRegressor(max_depth=len(features)+10, random_state=42)

# Entrenar un árbol con todo el conjunto de datos
arbol_importancia.fit(X=df_solar_filt2[features], y=df_solar_filt2[target])

In [ ]:
# Comprobamos que se obtiene un R^2 muy alto. Lo desesable es que sea 1.
y_pred_arbol = arbol_importancia.predict(X=df_solar_filt2[features])

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

# Métricas para evaluar la calidad del modelo
print('Mean Absolute Error:', mean_absolute_error(df_solar_filt2[target], y_pred_arbol))
print('Mean Absolute Percentage Error:', mean_absolute_percentage_error(df_solar_filt2[target], y_pred_arbol)*100)
print('Mean Squared Error:', mean_squared_error(df_solar_filt2[target], y_pred_arbol))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(df_solar_filt2[target], y_pred_arbol)))
print('R^2 coefficient of determination:', r2_score(df_solar_filt2[target], y_pred_arbol))

In [ ]:
# Importancia de cada variable en el árbol ajustado
arbol_importancia.feature_importances_

In [ ]:
# Crear un dataframe con los datos de importancia
importancia = pd.DataFrame(arbol_importancia.feature_importances_, index=features, columns=["Importancia"])

# Ordenamos los datos
importancia.sort_values(by=importancia.columns[0], ascending=False, inplace=True)
importancia

In [ ]:
importancia["imp_acum"] = importancia["Importancia"].cumsum()
importancia

In [ ]:
# Conjunto de variables más importantes
importancia.loc[importancia['imp_acum']<=0.90]

In [ ]:
variables = importancia.loc[importancia['imp_acum']>0.90].index.to_list()
print(variables)

Todas las variables tienen importancia, no eliminamos ninguna.

### Planteamiento del ejercicio de clasificación

# Hacemos una copia del tablón filtrado para no trabajar sobre original


In [ ]:
# Hacemos una copia del tablón filtrado para no trabajar sobre original
df_solpl = df_solar_filt2.copy()

In [ ]:
#Crearemos ahora una nueva columna llamada escenario
df_solpl["escenario"] = np.where(df_solpl["MODULE_TEMPERATURE"]<df_solpl["MODULE_TEMPERATURE"].mean(),"frio","calor")
df_solpl.drop(["MODULE_TEMPERATURE"],axis=1,inplace=True)
df_solpl

In [ ]:
df_solpl.describe()

In [ ]:
#graficar primer escenario
plt.figure(figsize=(10,5))
sns.violinplot(data=df_solpl, orient="h")
plt.show()

### **Paso 1.**  Obtención y preparación de datos

In [ ]:
# preparar los datos del modelo
# Variables independientes
X = df_solpl.drop(["escenario"],axis= 'columns')
# Variable dependiente
y = df_solpl["escenario"]

In [ ]:
# importamos libreria sklear
from sklearn.preprocessing import StandardScaler
# tipo de escalamiento y su ajuste
escalado = StandardScaler().fit(X)
dataset_normal = escalado.transform(X)
dataset_normal

In [ ]:
# Lo convertimos en un DataFrame, añadiendole sus etiquetas
X_normal = pd.DataFrame(dataset_normal, columns=X.columns)
print(type(X_normal))
X_normal

In [ ]:
X_normal.describe().round(4)

In [ ]:
# Graficar la distribución de los valores estandarizados
plt.figure(figsize=(10,5))
sns.violinplot(data=X_normal, orient="h")
plt.show()

### **Paso 2.**  Dividir el dataset en Training y Test

In [ ]:
# Separar los conjuntos de datos de entrenamiento (Training) y de prueba (Test) para las variables de entrada y salida
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_normal, y, test_size=0.2, random_state=42)


In [ ]:
# "test_size" representa la proporción del conjunto de datos a incluir en la división de Test
print(X_train.shape[0])
print(X_test.shape[0])
X_train.shape[0] + X_test.shape[0]

### **Paso 3.** Cargar y elegir el modelo de regresión logística

In [ ]:
# Importar el módulo que corresponde al algoritmo
from sklearn.linear_model import LogisticRegression

# Asignar el algoritmo que vamos a aplicar 
log_r = LogisticRegression(max_iter= 1000, random_state=42)

### **Paso 4.** Entrenar el modelo de regresión logística con los datos de entrenamiento

In [ ]:
# Entrenar el modelo
log_r.fit(X_train, y_train)

### **Paso 5.** Obtener las predicciones 

In [ ]:
# Calcular las predicciones con el conjunto de prueba
y_pred = log_r.predict(X_test)

In [ ]:
# Imprimir la salida del modelo (los niveles de calidad del aire)
print(y_pred)

### **Paso 6.** Evaluación del modelo a través de sus métricas

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

Clasificacion report

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

disp = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix(y_test, y_pred),
                               display_labels=log_r.classes_)
disp.plot()

plt.show()

In [ ]:
# Calculo del area bajo la curva ROC
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred)

In [ ]:
# Curva ROC
from sklearn.metrics import RocCurveDisplay

RocCurveDisplay.from_predictions(y_test, y_pred)

plt.show()